In [66]:
import pandas as pd
import numpy as np 
import torch
from transformers import *
from tqdm import tqdm 
import torch.nn as nn 
import torch.nn.functional as F 
from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler 
from tqdm.auto import tqdm 
import faiss 
from rank_bm25 import BM25Okapi

In [2]:
train = pd.read_csv("train.csv")

tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base")

questions, answers = [], [] 

for _, row in tqdm(train.iterrows()):
    for q_col in ["질문_1", "질문_2"]:
        for a_col in ["답변_1", "답변_2", "답변_3", "답변_4", "답변_5"]:
            questions.append(row[q_col])
            answers.append(row[a_col])

https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmph6vqwl2g


storing https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/e2d347346e3e13ea8d3e50ab2baef2cde9e7942cb05158cbd0effaa54af4e6e0.0763f758d6f4d3780831e69ac1702755a04a05bdbcb3e4f7692b6b546171ccb1
creating metadata file for /root/.cache/huggingface/transformers/e2d347346e3e13ea8d3e50ab2baef2cde9e7942cb05158cbd0effaa54af4e6e0.0763f758d6f4d3780831e69ac1702755a04a05bdbcb3e4f7692b6b546171ccb1
https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp_w1__6uh


storing https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/00ac7c2886f9d4555133877badce522b93b38439d90b0135d9b414cc1fafd167.34d17d2d06e0d29acc69761e3ddeced0dfdcf4cefa0aa81a1bb267a7dfdd5bcb
creating metadata file for /root/.cache/huggingface/transformers/00ac7c2886f9d4555133877badce522b93b38439d90b0135d9b414cc1fafd167.34d17d2d06e0d29acc69761e3ddeced0dfdcf4cefa0aa81a1bb267a7dfdd5bcb
https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpibr80kns


storing https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/e2eb4ad30139b806997f999b45c0a0d9ea38b14e0d97f42db852be137e061b1e.616843352d77fff459e989408eaacf1280dc39dcd346ff746aa3b3fbe6a123d9
creating metadata file for /root/.cache/huggingface/transformers/e2eb4ad30139b806997f999b45c0a0d9ea38b14e0d97f42db852be137e061b1e.616843352d77fff459e989408eaacf1280dc39dcd346ff746aa3b3fbe6a123d9
https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpmckcq2xw


storing https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/9bea998b48658e35dd618115a266f6c173183a9a4261fc6e40730d74c4b67899.e3640e465e51ce85d94923a0b396029ecc2e3e4c7764031eee57ab272637652d
creating metadata file for /root/.cache/huggingface/transformers/9bea998b48658e35dd618115a266f6c173183a9a4261fc6e40730d74c4b67899.e3640e465e51ce85d94923a0b396029ecc2e3e4c7764031eee57ab272637652d
loading file https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/00ac7c2886f9d4555133877badce522b93b38439d90b0135d9b414cc1fafd167.34d17d2d06e0d29acc69761e3ddeced0dfdcf4cefa0aa81a1bb267a7dfdd5bcb
loading file https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/e2eb4ad30139b806997f999b45c0a0d9ea38b14e0d97f42db852be137e061b1e.616843352d77fff459e989408eaacf1280dc39dcd346ff74

0it [00:00, ?it/s]

In [5]:
model_name = "monologg/kobigbird-bert-base"

answer_embedder = AutoModel.from_pretrained(model_name) 
device = torch.device('cuda') 

a_input_ids, a_attn_masks = [], [] 
for i in tqdm(range(len(questions)), position=0, leave=True):
    encoded_answer = tokenizer(answers[i], max_length=512, truncation=True, padding="max_length") 
    a_input_ids.append(encoded_answer["input_ids"]) 
    a_attn_masks.append(encoded_answer["attention_mask"])

https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp6gbcei1m


storing https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/3ff1f36a44e63a0ac32fcc55ff4c268a360e07ee22869bbc20ded21da8fdd596.4449f16b91f50859dc03ca5c81261c9952b3176fd389a7e99d067b33c0a8f3a1
creating metadata file for /root/.cache/huggingface/transformers/3ff1f36a44e63a0ac32fcc55ff4c268a360e07ee22869bbc20ded21da8fdd596.4449f16b91f50859dc03ca5c81261c9952b3176fd389a7e99d067b33c0a8f3a1
loading configuration file https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3ff1f36a44e63a0ac32fcc55ff4c268a360e07ee22869bbc20ded21da8fdd596.4449f16b91f50859dc03ca5c81261c9952b3176fd389a7e99d067b33c0a8f3a1
Model config BigBirdConfig {
  "_name_or_path": "monologg/kobigbird-bert-base",
  "architectures": [
    "BigBirdForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attention_type": "block_sparse",
  "block_size": 64,
  "bos_token_id": 5,
  "clas

storing https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/961f8c88dfa85a4b5bcdadb16abba817cf4cb6bf38ad0e2114249f4429efe451.d0cebe466881f586582c73d04be9d48ce3aafa4a491ec9898f4ea4b9e010ad41
creating metadata file for /root/.cache/huggingface/transformers/961f8c88dfa85a4b5bcdadb16abba817cf4cb6bf38ad0e2114249f4429efe451.d0cebe466881f586582c73d04be9d48ce3aafa4a491ec9898f4ea4b9e010ad41
loading weights file https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/961f8c88dfa85a4b5bcdadb16abba817cf4cb6bf38ad0e2114249f4429efe451.d0cebe466881f586582c73d04be9d48ce3aafa4a491ec9898f4ea4b9e010ad41
Some weights of the model checkpoint at monologg/kobigbird-bert-base were not used when initializing BigBirdModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dens

  0%|          | 0/6440 [00:00<?, ?it/s]

In [9]:
a_input_ids = torch.tensor(a_input_ids, dtype=int) 
a_attn_masks = torch.tensor(a_attn_masks, dtype=int) 

In [11]:
answer_chkpt = torch.load("answer_embedder_DPR.pt") 

answer_embedder.load_state_dict(answer_chkpt) 
answer_embedder.to(device)
answer_embedder.eval() 

answer_embeddings = [] 

batch_size = 16
answer_data = TensorDataset(a_input_ids, a_attn_masks) 
answer_sampler = SequentialSampler(answer_data) 
answer_dataloader = DataLoader(answer_data, sampler=answer_sampler, batch_size=batch_size) 

for batch in tqdm(answer_dataloader): 
    with torch.no_grad(): 
        batch = tuple(t.to(device) for t in batch)
        answer_input_ids, answer_attn_masks = batch 
        answer_embedding = answer_embedder(answer_input_ids, answer_attn_masks).pooler_output 
        answer_embedding = answer_embedding.detach().cpu()
        answer_embeddings.append(answer_embedding) 

  0%|          | 0/403 [00:00<?, ?it/s]

In [15]:
answer_embeddings = torch.cat(answer_embeddings, dim=0)

In [16]:
answer_embeddings.shape

torch.Size([6440, 768])

In [30]:
answer_embeddings = answer_embeddings.to(device)

In [23]:
answer_embeddings.shape

torch.Size([6440, 768])

In [18]:
test_question_embeddings = [] 
question_embedder = AutoModel.from_pretrained(model_name) 
question_chkpt = torch.load("question_embedder_DPR.pt")
question_embedder.to(device) 
question_embedder.load_state_dict(question_chkpt)
question_embedder.eval() 

test = pd.read_csv("test.csv")

loading configuration file https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3ff1f36a44e63a0ac32fcc55ff4c268a360e07ee22869bbc20ded21da8fdd596.4449f16b91f50859dc03ca5c81261c9952b3176fd389a7e99d067b33c0a8f3a1
Model config BigBirdConfig {
  "_name_or_path": "monologg/kobigbird-bert-base",
  "architectures": [
    "BigBirdForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attention_type": "block_sparse",
  "block_size": 64,
  "bos_token_id": 5,
  "classifier_dropout": null,
  "eos_token_id": 6,
  "gradient_checkpointing": false,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 4096,
  "model_type": "big_bird",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_random_blocks": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "rescal

In [20]:
test_questions = test["질문"].values 

q_input_ids, q_attn_masks = [], [] 
for i in tqdm(range(len(test_questions)), position=0, leave=True):
    encoded_question = tokenizer(test_questions[i], max_length=512, truncation=True, padding="max_length") 
    q_input_ids.append(encoded_question["input_ids"]) 
    q_attn_masks.append(encoded_question["attention_mask"])

  0%|          | 0/130 [00:00<?, ?it/s]

In [21]:
q_input_ids = torch.tensor(q_input_ids, dtype=int) 
q_attn_masks = torch.tensor(q_attn_masks, dtype=int) 

q_input_ids.shape, q_attn_masks.shape 

(torch.Size([130, 512]), torch.Size([130, 512]))

In [42]:
top_answers = [] 

for i in tqdm(range(len(q_input_ids))): 
    cur_input_ids = q_input_ids[i] 
    cur_attn_mask = q_attn_masks[i] 
    cur_input_ids = torch.reshape(cur_input_ids, (-1, 512)) 
    cur_attn_mask = torch.reshape(cur_attn_mask, (-1, 512)) 
    cur_input_ids = cur_input_ids.to(device) 
    cur_attn_mask = cur_attn_mask.to(device)
    with torch.no_grad():
        q_emb = question_embedder(cur_input_ids, cur_attn_mask).pooler_output 
        dot_prod_scores = torch.matmul(q_emb, torch.transpose(answer_embeddings, 0, 1))
        rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()  
        top_answers.append(answers[rank[0]])

  0%|          | 0/130 [00:00<?, ?it/s]

In [44]:
submission = pd.read_csv("sample_submission.csv")

In [46]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(top_answers)
pred_embeddings.shape

loading configuration file /root/.cache/torch/sentence_transformers/sentence-transformers_distiluse-base-multilingual-cased-v1/config.json
Model config DistilBertConfig {
  "_name_or_path": "/root/.cache/torch/sentence_transformers/sentence-transformers_distiluse-base-multilingual-cased-v1/",
  "activation": "gelu",
  "architectures": [
    "DistilBertModel"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.3",
  "vocab_size": 119547
}

loading weights file /root/.cache/torch/sentence_transformers/sentence-transformers_distiluse-base-multilingual-cased-v1/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertModel.

All the weights of D

(130, 512)

In [47]:
submission.iloc[:,1:] = pred_embeddings
submission.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.003907,0.026506,0.002935,-0.020368,0.093865,-0.055855,0.022940,0.021612,-0.036171,...,-0.029012,-0.028996,0.036807,-0.019275,-0.000355,0.093617,-0.016542,-0.036665,-0.031433,0.060282
1,TEST_001,-0.034920,-0.051659,0.034640,0.006939,0.059443,-0.005048,-0.016672,0.024080,0.012391,...,-0.016735,-0.024383,0.031203,-0.047914,-0.048928,0.049996,0.012761,-0.035183,0.058803,-0.008451
2,TEST_002,0.014635,0.011680,-0.106070,0.010486,0.090208,-0.080892,-0.007853,-0.002600,0.010746,...,-0.040023,-0.054669,0.017772,-0.049280,0.015645,0.013285,-0.031572,-0.012180,-0.015811,0.101507
3,TEST_003,-0.006627,0.028780,-0.041766,0.038018,0.047464,-0.053030,-0.024172,-0.001850,-0.024162,...,-0.059843,-0.041408,0.010751,-0.015097,-0.021498,0.066948,-0.013049,-0.000514,-0.026622,0.071312
4,TEST_004,-0.059788,-0.020483,0.016567,-0.035316,0.002403,-0.014444,0.090898,0.006764,0.009712,...,-0.026803,0.013162,-0.000635,0.061460,-0.013816,-0.008346,0.008502,0.044158,0.045571,0.010090


In [48]:
# 리더보드 제출을 위한 csv파일 생성
submission.to_csv('raw_DPR.csv', index=False)


## Concatenation of multiple answers (top 50)

In [87]:
top_answers = [] 

for i in tqdm(range(len(q_input_ids))): 
    cur_input_ids = q_input_ids[i] 
    cur_attn_mask = q_attn_masks[i] 
    cur_input_ids = torch.reshape(cur_input_ids, (-1, 512)) 
    cur_attn_mask = torch.reshape(cur_attn_mask, (-1, 512)) 
    cur_input_ids = cur_input_ids.to(device) 
    cur_attn_mask = cur_attn_mask.to(device)
    with torch.no_grad():
        q_emb = question_embedder(cur_input_ids, cur_attn_mask).pooler_output 
        dot_prod_scores = torch.matmul(q_emb, torch.transpose(answer_embeddings, 0, 1))
        rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()  
        top50_answers = [answers[rank[j]] for j in range(50)] 
        top50_answers = np.unique(top50_answers)
        top_answers.append(top50_answers) 

  0%|          | 0/130 [00:00<?, ?it/s]

In [88]:
bm25_filtered = [] 

for i in tqdm(range(len(test_questions))):
    tokenized_corpus = [tokenizer.tokenize(doc) for doc in top_answers[i]]
    tokenized_query = tokenizer.tokenize(test_questions[i])
    bm25 = BM25Okapi(tokenized_corpus)
    doc_scores = bm25.get_scores(tokenized_query)
    top_doc_indices = np.argsort(doc_scores)[::-1][:5]
    top_docs = [top_answers[i][index] for index in top_doc_indices]
    concatenated_answer = " ".join(top_docs)
    bm25_filtered.append(concatenated_answer)

  0%|          | 0/130 [00:00<?, ?it/s]

In [99]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(bm25_filtered)
pred_embeddings.shape

loading configuration file /root/.cache/torch/sentence_transformers/sentence-transformers_distiluse-base-multilingual-cased-v1/config.json
Model config DistilBertConfig {
  "_name_or_path": "/root/.cache/torch/sentence_transformers/sentence-transformers_distiluse-base-multilingual-cased-v1/",
  "activation": "gelu",
  "architectures": [
    "DistilBertModel"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.3",
  "vocab_size": 119547
}

loading weights file /root/.cache/torch/sentence_transformers/sentence-transformers_distiluse-base-multilingual-cased-v1/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertModel.

All the weights of D

(130, 512)

In [100]:
submission = pd.read_csv("sample_submission.csv") 
submission.iloc[:,1:] = pred_embeddings
submission.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,-0.009091,0.051201,0.053451,-0.006502,0.082316,-0.047824,0.015298,-0.003625,-0.052473,...,-0.050290,-0.047070,0.027378,-0.060718,0.026424,0.072346,0.027770,-0.052403,-0.003980,0.080497
1,TEST_001,0.001539,-0.014106,0.006531,0.013342,0.069398,-0.013717,-0.008829,-0.023080,-0.008041,...,-0.025134,-0.016303,0.043635,-0.045904,-0.025309,0.035036,-0.005466,-0.009464,0.036712,-0.004162
2,TEST_002,0.032605,-0.081564,-0.060735,0.009520,0.096750,0.006504,0.027207,-0.012308,0.016761,...,-0.037246,-0.021570,0.024303,-0.029671,0.014703,0.041096,-0.014370,-0.014931,0.010610,0.056091
3,TEST_003,0.028997,-0.004509,-0.019172,0.027558,0.082488,-0.047887,-0.058942,0.023962,0.018857,...,-0.031114,0.018074,0.069288,-0.063136,0.018898,0.004010,0.007449,-0.018820,-0.026087,0.058256
4,TEST_004,0.014048,-0.003082,-0.015159,-0.020049,0.104739,-0.020299,0.054620,0.043846,-0.014219,...,-0.003134,-0.026959,0.025491,0.006445,-0.036952,0.005618,0.003694,0.036506,-0.006303,0.050108


In [93]:
submission = pd.read_csv("sample_submission.csv") 
submission.iloc[:,1:] = pred_embeddings
submission.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,-0.009091,0.051201,0.053451,-0.006502,0.082316,-0.047824,0.015298,-0.003625,-0.052473,...,-0.050290,-0.047070,0.027378,-0.060718,0.026424,0.072346,0.027770,-0.052403,-0.003980,0.080497
1,TEST_001,0.001539,-0.014106,0.006531,0.013342,0.069398,-0.013717,-0.008829,-0.023080,-0.008041,...,-0.025134,-0.016303,0.043635,-0.045904,-0.025309,0.035036,-0.005466,-0.009464,0.036712,-0.004162
2,TEST_002,0.032605,-0.081564,-0.060735,0.009520,0.096750,0.006504,0.027207,-0.012308,0.016761,...,-0.037246,-0.021570,0.024303,-0.029671,0.014703,0.041096,-0.014370,-0.014931,0.010610,0.056091
3,TEST_003,0.028997,-0.004509,-0.019172,0.027558,0.082488,-0.047887,-0.058942,0.023962,0.018857,...,-0.031114,0.018074,0.069288,-0.063136,0.018898,0.004010,0.007449,-0.018820,-0.026087,0.058256
4,TEST_004,0.014048,-0.003082,-0.015159,-0.020049,0.104739,-0.020299,0.054620,0.043846,-0.014219,...,-0.003134,-0.026959,0.025491,0.006445,-0.036952,0.005618,0.003694,0.036506,-0.006303,0.050108


In [94]:
submission.to_csv("DPR_BM25_top5_concatenated.csv", index=False)